# Jetson ORT TensorRT EP Diagnose

This notebook helps you find why TensorRT EP is not active on Jetson.
It collects environment info, verifies TensorRT libraries, and tries to create an ONNX Runtime session
with detailed logging.

**Tip:** If you already imported `onnxruntime` in this kernel, restart the kernel and run from the top
so logging settings take effect.


In [ ]:
import os
import sys
import platform
from pathlib import Path

print('Python:', sys.version.replace('', ' '))
print('Platform:', platform.platform())
print('Machine:', platform.machine())
print('CWD:', Path.cwd())

env_keys = [
    'LD_LIBRARY_PATH',
    'PATH',
    'CUDA_VISIBLE_DEVICES',
    'ORT_LOG_SEVERITY_LEVEL',
    'ORT_LOG_VERBOSITY_LEVEL',
]
for key in env_keys:
    value = os.environ.get(key, '')
    print(f'{key}: {value}')


In [ ]:
# Optional: check Python packages and their versions
def _try_import(name):
    try:
        module = __import__(name)
        version = getattr(module, '__version__', 'unknown')
        print(f'{name} version:', version)
        return module
    except Exception as exc:
        print(f'{name} import failed:', exc)
        return None

_try_import('onnxruntime')
_try_import('tensorrt')
_try_import('torch')


In [ ]:
# Check TensorRT shared libraries
import ctypes

libs = [
    'libnvinfer.so',
    'libnvinfer_plugin.so',
    'libnvonnxparser.so',
]
for lib in libs:
    try:
        ctypes.CDLL(lib)
        print(lib, 'OK')
    except OSError as exc:
        print(lib, 'FAIL', exc)


## Set ORT logging and import onnxruntime
If you already imported onnxruntime in this kernel, restart and run from the top.


In [ ]:
import os
import sys

if 'onnxruntime' in sys.modules:
    print('onnxruntime already imported. Restart the kernel and run from the top for logging to work.')
else:
    os.environ['ORT_LOG_SEVERITY_LEVEL'] = '0'
    os.environ['ORT_LOG_VERBOSITY_LEVEL'] = '1'
    import onnxruntime as ort
    print('ORT version:', ort.__version__)
    print('Available providers:', ort.get_available_providers())


## Model path and input shape
Edit the model path if needed.


In [ ]:
from pathlib import Path

model_path = Path('yolo26n_my.onnx')
print('Model exists:', model_path.exists(), model_path.resolve())

if model_path.exists():
    import onnxruntime as ort
    sess = ort.InferenceSession(str(model_path), providers=['CUDAExecutionProvider'])
    inp = sess.get_inputs()[0]
    print('Input name:', inp.name)
    print('Input shape:', inp.shape)
    print('Input type:', getattr(inp, 'type', None))


## Try TensorRT EP with verbose logging
This should show any TensorRT initialization failures in the output.


In [ ]:
import onnxruntime as ort

trt_cache = Path('trt_cache')
trt_cache.mkdir(parents=True, exist_ok=True)

providers = [
    (
        'TensorrtExecutionProvider',
        {
            'trt_fp16_enable': True,
            'trt_engine_cache_enable': True,
            'trt_engine_cache_path': str(trt_cache),
            'trt_max_workspace_size': 2 * 1024 * 1024 * 1024,
        },
    ),
    ('CUDAExecutionProvider', {'device_id': 0}),
]

so = ort.SessionOptions()
so.log_severity_level = 0
so.log_verbosity_level = 1

try:
    sess = ort.InferenceSession(str(model_path), sess_options=so, providers=providers)
    print('Actual providers:', sess.get_providers())
except Exception as exc:
    print('Session creation failed:', exc)


## CUDA-only baseline (for comparison)
If TRT fails, this confirms CUDA EP works.


In [ ]:
import time
import numpy as np

sess = ort.InferenceSession(str(model_path), providers=['CUDAExecutionProvider'])
inp = sess.get_inputs()[0]
shape = [d if isinstance(d, int) else 1 for d in inp.shape]
dummy = np.random.rand(*shape).astype(np.float32)

# warmup
for _ in range(5):
    sess.run(None, {inp.name: dummy})

iters = 50
start = time.perf_counter()
for _ in range(iters):
    sess.run(None, {inp.name: dummy})
end = time.perf_counter()
avg_ms = (end - start) / iters * 1000
print(f'CUDA EP avg latency: {avg_ms:.3f} ms')


## Interpretation
- If TRT EP is available but not active, the logs above will usually show missing libraries or version mismatch.
- If CUDA EP works and TRT EP fails, the issue is specific to TensorRT integration (not the model).
- If both fail, the ONNX model or ORT install is broken.
